# Transition Integration

In this notebook we integrate 2d well with a high noise in order to observe transitions

In [1]:
# Standard Package imports
import numpy as np
import numpy.linalg as la
import numpy.random as rm
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# from fancyWell import *

## Integration Setup

In [7]:
def grad_V(x):
    return np.array([x[0]*(x[0]**2 -1), 2 * x[1]])

R = np.array([[0, -1], [1, 0]]) # 90 degree rotation matrix

In [21]:
# Parameter Choise

eps = 0.1 
alphas = [0., 0.25, 0.5, 1.]
dt = 0.01
tf = 50
time = np.arange(0, tf, dt)

In [22]:
def drift(x, p):
    alpha, eps = p
    return - (np.eye(2) + alpha * R) @ grad_V(x)

def euler_maruyama(x0, t, p, timer=False):
    alpha, eps = p
    N = len(t)
    x = np.zeros(np.append(N, x0.shape))
    x[0] = x0
    for i in tqdm(range(N-1), disable=not timer):
        dt = t[i+1]-t[i]
        dWt = rm.normal(0, np.sqrt(dt), 2)
        x[i+1] = x[i] + drift(x[i], p) * dt + np.sqrt(eps) * dWt
    return x

In [14]:
cold_point = np.array([-1, 0])
hot_point = np.array([1, 0])
saddle = np.array([0, 0])

## Running Ensemble of Stochastic Integrations

In [17]:
from joblib import Parallel, delayed
import multiprocessing

In [20]:
num_cores = multiprocessing.cpu_count()

In [ ]:
def cold_ensemble_run(p):
    cold_ensemble = []
    alpha, eps = p
    cold_ensemble.append(euler_maruyama(cold_point, time, [alpha, eps]))

    
    

In [ ]:
Parallel(n_jobs=num_cores)(delayed(manifold_angle)(CLV, data.k_d) for CLV in tqdm(inputs))


In [16]:
# Integration 

cold_ensemble = []
hot_ensemble = []
ensemble_size = int(1.e4)

for i in tqdm(range(ensemble_size)):
    cold_ensemble.append(euler_maruyama(cold_point, time, [alpha, eps]))
    
for i in tqdm(range(ensemble_size)):
    hot_ensemble.append(euler_maruyama(hot_point, time, p))

KeyboardInterrupt: 

In [ ]:
# Putting Results in xarray dataset and Saving

dims = ['x', 'y']
coords = [time]
attrs = {'alpha': alpha, 'eps': eps}

# Cold Ensemble
data = np.asarray(cold_ensemble)
x_data = xr.DataArray(data[:, :, 0], coords = {'realisation':np.arange(len(cold_ensemble)) + 1,
                                    'time': time},
                    dims = ['realisation','time'], attrs=attrs, name='x')
y_data = xr.DataArray(data[:, :, 1], coords = {'realisation':np.arange(len(cold_ensemble)) + 1,
                                    'time': time},
                    dims = ['realisation','time'], attrs=attrs, name='y')
cold_ensemble_ds = xr.merge([x_data, y_data])
cold_ensemble_ds.attrs = attrs

# Hot Ensemble

data = np.asarray(hot_ensemble)
x_data = xr.DataArray(data[:, :, 0], coords = {'realisation':np.arange(len(hot_ensemble)) + 1,
                                    'time': time},
                    dims = ['realisation','time'], attrs=attrs, name='x')
y_data = xr.DataArray(data[:, :, 1], coords = {'realisation':np.arange(len(hot_ensemble)) + 1,
                                    'time': time},
                    dims = ['realisation','time'], attrs=attrs, name='y')
hot_ensemble_ds = xr.merge([x_data, y_data])
hot_ensemble_ds.attrs = attrs